# 1_ Numpy Implementation of Logistic Regression

This notebook is to explain the implementation of logistic regression using only **Numpy** module. I will divide this problem into multiple sub-steps and explain each thing separately. 

Data used here is IRIS dataset available in sklearn module. Labels are converted into a binary variable where only label: 0 (Setosa) has been used as target for training and testing.

In [1]:
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = np.array([1 if i == 0 else 0 for i in iris.target]).reshape(-1, 1)
print("Shape of Feature Space : {}".format(X.shape))
print("Shape of Target Space: {}".format(y.shape))

Shape of Feature Space : (150, 4)
Shape of Target Space: (150, 1)


In above, label column has been explicitly reshaped into a column vector. In **Numpy**, usually scalar values are represented in (m, ) shape which can sometime generate weird results when passed to a function. It's a good practice to have explicitly defined shapes for all vectors wherever possible.

Also, since this is a small dataset, train and test set are simply created by choosing first 120 and last 30 rows. However, for larger datasets it is recommended to use a random selection method.

In [2]:
train_X, test_X = X[:120], X[120:]
train_y, test_y = y[:120], y[120:]

## 1.1_ General Knowledge

<img src="LR.png" style="width:650px;height:400px;">

Logistic regression is a model which uses a logistic function to classify a binary dependent variable. 

**Mathematical expression of the algorithm**:

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{4}$$

# 2_ Building the parts of our algorithm

The main steps are:

1. Define the model structure (such as number of input features) 
2. Initialize the model's parameters (weight matrix and bias)
3. Loop:
    - Calculate current loss (forward propagation)
    - Calculate current gradient (backward propagation)
    - Update parameters (gradient descent)
    
## 2.1_ Helper Functions

**Sigmoid**

Purpose: Computes sigmoid of input z

Arguments :

        z -- a scalar or numpy array of any size
        
Return :

        s -- sigmoid value of input

In [3]:
def sigmoid(z):
        
    s = 1/(1 + np.exp(-z))
    return s

print("Sigmoid of Zero: {}".format(sigmoid(0)))
print("Sigmoid of Ten: {}".format(sigmoid(10)))

Sigmoid of Zero: 0.5
Sigmoid of Ten: 0.9999546021312976


## 2.2_ Initializing Parameters

**initialize**

Purpose : creates a vector of random numbers of shape (dim, 1) for weights and initializes bias with 0.

Arguments :

        dim -- size of the w vector we want (or number of parameters in this case)

Returns :

        w -- initialized vector of shape (dim, 1)
  
        b -- initialized scalar (corresponds to the bias)

In [4]:
def initialize(dim):
    
    w = np.random.randn(dim, 1)
    b = 0
    return w, b

In [5]:
w, b = initialize(4)
print("w = " + str(w))
print("b = " + str(b))

w = [[ 0.29868666]
 [-1.3379613 ]
 [ 0.64811609]
 [ 1.15116898]]
b = 0


## 2.3_ Calculate Gradients and Loss

**propagate**

Purpose : Calcualte cost function and its gradient for each pass of data

Arguments :

        w -- weights, a numpy array
        b -- bias, a scalar
        X -- features
        Y -- label

Return : 

        cost -- negative log-likelihood cost for logistic regression
        dw -- gradient of the loss with respect to w, thus same shape as w
        db -- gradient of the loss with respect to b, thus same shape as b 

In [6]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
 
    m = X.shape[0]
    A = sigmoid(np.dot(X, w) + b)  # compute logistic value
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))  # compute negative logloss 
    
    ### Calculate gradient ###
    dw = (1 / m) * np.dot(X.T, (A - Y))
    db = (1 / m) * np.sum(A - Y)
    
    ## Return values ##
    grads = {"dw": dw, "db": db}
    
    return grads, cost

In [7]:
propagate(w, b, train_X, train_y)

({'dw': array([[1.52741652],
         [0.29162463],
         [2.01788092],
         [0.7494527 ]]), 'db': 0.18372204544164408}, 2.5441962009194365)

## 2.4_ Optimization

**optimize**

Purpose : optimizes w and b by running a gradient descent algorithm

Arguments : 
        
        w -- weights, a numpy array
        b -- bias, a scalar
        X -- features
        Y -- label
        num_iterations -- number of times data should be passed to minimize the loss
        learning_rate -- learning rate of the gradient descent update rule
        print_cost -- True to print the loss every 100 steps

Returns : 
        
        params -- dictionary containing the weights w and bias b
        grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
        costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve

In [8]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        # Cost and gradient calculated in each iteration
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule
        w = w - learning_rate * dw  
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration {} : {}".format(i, cost))
    
    params = {"w": w, "b": b}
    
    return params, costs

In [9]:
optimize(w, b, train_X, train_y, 100, 0.5)

({'w': array([[ 0.9655611 ],
         [ 1.11827664],
         [-3.15882581],
         [-0.54523699]]), 'b': 0.442167435844594}, [2.5441962009194365])

## 2.5_ Predict

**predict**

Purpose : Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)

Arguments : 
        
        w -- weights, a numpy array
        b -- bias, a scalar
        X -- features

Returns : 

        pred -- a numpy array (vector) containing all predictions (0/1) based on a threshold
        

In [10]:
def predict(w, b, X, threshold = 0.5):
        
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    # Compute vector "A" predicting the probabilities
    A = sigmoid(np.dot(X, w) + b)
    
    for i in range(A.shape[0]):
        # Convert probabilities to actual predictions
        pred[i, 0] = 1 if A[i, 0] > threshold else 0
    
    return pred

# 3_ Merge all into a model

**model** 

Purpose : Builds the logistic regression model by calling functions defined before

Arguments :
       
        X_train -- training set represented by a numpy array 
        Y_train -- training labels represented by a numpy array (vector) 
        X_test -- test set represented by a numpy array 
        Y_test -- test labels represented by a numpy array (vector) 
        num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
        learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
        print_cost -- Set to true to print the cost every 100 iterations
        
Returns :
        
        d -- dictionary containing information about the model
        

In [11]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):

    # initialize parameters 
    w, b = initialize(X_train.shape[1])

    # Gradient descent
    parameters, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples 
    pred_test = predict(w, b, X_test)
    pred_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(pred_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(pred_test - Y_test)) * 100))
  
    d = {"costs": costs,
         "Y_prediction_test": pred_test, 
         "Y_prediction_train" : pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [12]:
d = model(train_X, train_y, test_X, test_y, print_cost = True)

Cost after iteration 0 : 5.169811708647467
Cost after iteration 100 : 0.012362593931101125
Cost after iteration 200 : 0.0069643139305350064
Cost after iteration 300 : 0.004914024031873643
Cost after iteration 400 : 0.0038230708049046933
Cost after iteration 500 : 0.0031419440833102253
Cost after iteration 600 : 0.002674535815169117
Cost after iteration 700 : 0.0023330344901463062
Cost after iteration 800 : 0.0020721163935097536
Cost after iteration 900 : 0.0018659714153750077
Cost after iteration 1000 : 0.001698798656718109
Cost after iteration 1100 : 0.0015603714344647399
Cost after iteration 1200 : 0.0014437719361550135
Cost after iteration 1300 : 0.0013441493591226908
Cost after iteration 1400 : 0.0012579992626569171
Cost after iteration 1500 : 0.0011827253146116166
Cost after iteration 1600 : 0.0011163620886007679
Cost after iteration 1700 : 0.0010573937616868863
Cost after iteration 1800 : 0.0010046320641398823
Cost after iteration 1900 : 0.0009571320252150256
train accuracy: 100.

# 4_ Conclusion

This work is not a complete method for Logistic Regression. It has been developed only for understanding the most common and essential operations involved in an LR model. Few things which are still lacking in this implementation are:

1. Calculation of p-values or any variable static to understand the importance of it in the model
2. Calculation of testing loss in order to check for over/under fitting
3. Integration of any regularization method to help in building a robust model